In [1]:
!pip install yfinance
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=6cdb9de4f6f5b025e710af26cc9d2e82daa01ed0d645a66704a2c68a00a08e08
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [7]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from google.colab import files

# Define a list of stock symbols
stock_symbols = [
    "FL", "MRNA", "BABA", "PFE", "WBA", "AAL", "BNTX", "PYPL", "MMM", "DIS",
    "CVS", "VZ", "NKE", "TRIP", "UAL", "VOD", "T", "QCOM", "LEVI", "GM", "C",
    "LUV", "TSM", "GTLB", "BAC", "F", "SCHW", "GS", "WFC", "TGT"
]  # Add more tickers as needed

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()
def generate_signals(data):
    signals = pd.DataFrame(index=data.index)
    signals['MA_Crossover_Signal'] = 'Hold'
    signals['MACD_Signal'] = 'Hold'
    signals['RSI_Signal'] = 'Hold'
    signals['Bollinger_Signal'] = 'Hold'
    signals['Supertrend_Signal'] = 'Hold'

    # MA50 vs MA200
    signals.loc[data['MA50'] > data['MA200'], 'MA_Crossover_Signal'] = 'Buy'
    signals.loc[data['MA50'] < data['MA200'], 'MA_Crossover_Signal'] = 'Sell'

    # MACD
    signals.loc[data['MACD'] > data['Signal_Line'], 'MACD_Signal'] = 'Buy'
    signals.loc[data['MACD'] < data['Signal_Line'], 'MACD_Signal'] = 'Sell'

    # RSI
    signals.loc[data['RSI'] < 30, 'RSI_Signal'] = 'Buy'
    signals.loc[data['RSI'] > 70, 'RSI_Signal'] = 'Sell'

    # Bollinger Bands
    signals.loc[data['Price'] < data['Lower_Band'], 'Bollinger_Signal'] = 'Buy'
    signals.loc[data['Price'] > data['Upper_Band'], 'Bollinger_Signal'] = 'Sell'

    # Supertrend
    signals.loc[data['Price'] > data['Supertrend'], 'Supertrend_Signal'] = 'Buy'
    signals.loc[data['Price'] < data['Supertrend'], 'Supertrend_Signal'] = 'Sell'

    return signals

for stock_symbol in stock_symbols:

    # Fetch historical stock data using yfinance
    stock_data = yf.download(stock_symbol, period="1y")  # Adjust the period as needed

    # Calculate 50-day and 200-day Simple Moving Averages (SMA) using pandas_ta
    sma_50 = ta.sma(stock_data["Close"], length=50)   # 50-day SMA
    sma_200 = ta.sma(stock_data["Close"], length=200)  # 200-day SMA

    # Calculate Relative Strength Index (RSI) using pandas_ta
    rsi = ta.rsi(stock_data["Close"], length=14)  # 14-day RSI

    # Calculate MACD using pandas_ta
    macd_data = ta.macd(stock_data["Close"], fast=12, slow=26, signal=9)

    # Calculate Bollinger Bands using pandas_ta
    bollinger_bands = ta.bbands(stock_data["Close"], length=20, std=2)  # 20-day Bollinger Bands with 2 standard deviations

    # Calculate Supertrend using pandas_ta
    supertrend = ta.supertrend(stock_data["High"], stock_data["Low"], stock_data["Close"], length=7, multiplier=3)

    # Get the most recent values for each indicator
    current_sma_50 = sma_50.iloc[-1]
    current_sma_200 = sma_200.iloc[-1]
    current_rsi = rsi.iloc[-1]
    current_macd = macd_data['MACD_12_26_9'].iloc[-1]  # Extract only the MACD Line value
    current_signal_line = macd_data['MACDs_12_26_9'].iloc[-1]  # Signal Line
    current_bb = bollinger_bands['BBM_20_2.0'].iloc[-1]
    upper_bb = bollinger_bands['BBU_20_2.0'].iloc[-1]
    lower_bb = bollinger_bands['BBL_20_2.0'].iloc[-1]
    current_super_trend = supertrend['SUPERT_7_3.0'].iloc[-1]
    current_close_price = stock_data["Close"].iloc[-1]

    # Create a Ticker object for the specified ticker symbol
    stock = yf.Ticker(stock_symbol)

    # Get additional financial data
    try:
        info = stock.info
        name = info.get('longName', 'N/A')

        # Create a new DataFrame with the financial data
        financial_data = {
            "Symbol": [stock_symbol],
            "Name": [name],
        }

        financial_data_df = pd.DataFrame(financial_data)

        # Create a new DataFrame with the current values for this stock
        data = {
            "MA50": [current_sma_50],
            "MA200": [current_sma_200],
            "RSI": [current_rsi],
            "MACD": [current_macd],  # Include only the MACD Line
            "Signal_Line": [current_signal_line],  # Include the Signal Line
            "Lower_Band": [lower_bb],
            "Bollinger Bands": [current_bb],
            "Upper_Band": [upper_bb],
            "Supertrend": [current_super_trend],
            "Price": [current_close_price]
        }

        data_df = pd.DataFrame(data)
        signals_df = generate_signals(data_df)

        # Concatenate the financial data DataFrame with the result DataFrame
        combined_df = pd.concat([financial_data_df, data_df, signals_df], axis=1)

        # Concatenate the combined DataFrame with the result DataFrame
        result_df = pd.concat([result_df, combined_df], ignore_index=True)

    except Exception as e:
        print(f"An error occurred for {stock_symbol}: {str(e)}")

# Save the result_df to a CSV file
result_df.to_csv("stock_data.csv", index=False)
# Download the CSV file
files.download("stock_data.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>